In [ ]:
%%capture
!git clone https://github.com/promptslab/Promptify.git
!pip3 install openai

<h2>Features </h2>
<ul>
  <li> NER in 2 lines of code</li>
  <li> Easily add one shot, two shot, or few shot examples to the prompt</li>
  <li> Output always provided as a Python object (e.g. list, dictionary) for easy parsing and filtering</li>
  <li> Custom examples and samples can be easily added to the prompt</li>
  <li> Optimized prompts to reduce OpenAI token costs (coming soon)</li>
</ul>




### Define any LLM model

In [ ]:
%cd /content/Promptify

import json
from promptify.models.nlp.openai_model import OpenAI
from promptify.prompts.nlp.prompter import Prompter
from pprint import pprint
from IPython.display import Markdown, display
from IPython.core.display import display, HTML


# Define the API key for the OpenAI model
api_key  = ""


# Create an instance of the OpenAI model, Currently supporting Openai's all model, In future adding more generative models from Hugginface and other platforms
model = OpenAI(api_key)
nlp_prompter = Prompter(model)


# Example sentence for demonstration
sent = "The patient is a 93-year-old female with a medical history of chronic right hip pain, osteoporosis, hypertension, depression, and chronic atrial fibrillation admitted for evaluation and management of severe nausea and vomiting and urinary tract infection"
print(sent)

/content/Promptify
The patient is a 93-year-old female with a medical history of chronic right hip pain, osteoporosis, hypertension, depression, and chronic atrial fibrillation admitted for evaluation and management of severe nausea and vomiting and urinary tract infection


### 1: Named Entity Recognition (NER) Example in 2 Lines of code


In [ ]:
# Named Entity Recognition with No labels, no description, no oneshot, no examples
# Simple prompt with instructions
# domain name gives more info to model for better result generation, the parameter is optional
# Output will be python object -> [ {'E' : Entity Name, 'T': Type of Entity } ]


result = nlp_prompter.fit('ner.jinja',
                          domain      = 'medical',
                          text_input  = sent, 
                          labels      = None)

# Output
pprint(eval(result['text']))



The patient is a 93-year-old female with a medical history of chronic right hip pain, osteoporosis, hypertension, depression, and chronic atrial fibrillation admitted for evaluation and management of severe nausea and vomiting and urinary tract infection






[{'E': '93-year-old', 'T': 'Age'},
 {'E': 'chronic right hip pain', 'T': 'Medical Condition'},
 {'E': 'osteoporosis', 'T': 'Medical Condition'},
 {'E': 'hypertension', 'T': 'Medical Condition'},
 {'E': 'depression', 'T': 'Medical Condition'},
 {'E': 'chronic atrial fibrillation', 'T': 'Medical Condition'},
 {'E': 'severe nausea and vomiting', 'T': 'Symptom'},
 {'E': 'urinary tract infection', 'T': 'Medical Condition'},
 {'Branch': 'Internal Medicine', 'Group': 'Geriatrics'}]


In [ ]:
# You can also return the output with start and end position of the tokens. 
# For that use the output format like this -> [{'E': '93-year-old', 'T': 'Age', 'S' : Start Position, 'En' : End Position}]

###2 : Named Entity Recognition (NER) with Custom Tags/Entities



In [ ]:
# Case : 2
# If want to perform NER with custom tags only (hangling out-of-bounds prediction) prompt


result = nlp_prompter.fit('ner.jinja',
                          domain      = 'medical',
                          text_input  = sent, 
                          labels      = ["SYMPTOM", "DISEASE"])

# Output
pprint(eval(result['text']))



The patient is a 93-year-old female with a medical history of chronic right hip pain, osteoporosis, hypertension, depression, and chronic atrial fibrillation admitted for evaluation and management of severe nausea and vomiting and urinary tract infection






[{'E': 'nausea and vomiting', 'T': 'SYMPTOM'},
 {'E': 'urinary tract infection', 'T': 'SYMPTOM'},
 {'E': 'osteoporosis', 'T': 'DISEASE'},
 {'E': 'hypertension', 'T': 'DISEASE'},
 {'E': 'depression', 'T': 'DISEASE'},
 {'E': 'chronic atrial fibrillation', 'T': 'DISEASE'},
 {'branch': 'Medical', 'group': 'History'}]


###3 : Named Entity Recognition (NER) with One shot Example

In [ ]:
# Case : 3
# If want to perform NER wit one shot example adding by default
# Observe The changes in the model's output
# the example format should be -> [ [Text, [{'E' : Entity Name, 'T': Type of Entity }]] ]

one_shot = "Leptomeningeal metastases (LM) occur in patients with breast cancer (BC) and lung cancer (LC). The cerebrospinal fluid (CSF) tumour microenvironment (TME) of LM patients is not well defined at a single-cell level. We did an analysis based on single-cell RNA sequencing (scRNA-seq) data and four patient-derived CSF samples of idiopathic intracranial hypertension (IIH)"
one_shot = [[one_shot, [{'E': 'DISEASE', 'W': 'Leptomeningeal metastases'}, {'E': 'DISEASE', 'W': 'breast cancer'}, {'E': 'DISEASE', 'W': 'lung cancer'}, {'E': 'BIOMARKER', 'W': 'cerebrospinal fluid'}, {'E': 'DISEASE', 'W': 'tumour microenvironment'}, {'E': 'TEST', 'W': 'single-cell RNA sequencing'}, {'E': 'DISEASE', 'W': 'idiopathic intracranial hypertension'}]]]


result = nlp_prompter.fit('ner.jinja',
                          domain      = 'medical',
                          text_input  = sent,
                          examples    = one_shot,
                          labels      = ["SYMPTOM", "DISEASE"])


pprint(eval(result['text']))



The patient is a 93-year-old female with a medical history of chronic right hip pain, osteoporosis, hypertension, depression, and chronic atrial fibrillation admitted for evaluation and management of severe nausea and vomiting and urinary tract infection






[[{'E': 'SYMPTOM', 'W': 'chronic right hip pain'},
  {'E': 'DISEASE', 'W': 'osteoporosis'},
  {'E': 'DISEASE', 'W': 'hypertension'},
  {'E': 'SYMPTOM', 'W': 'depression'},
  {'E': 'DISEASE', 'W': 'chronic atrial fibrillation'},
  {'E': 'SYMPTOM', 'W': 'severe nausea and vomiting'},
  {'E': 'DISEASE', 'W': 'urinary tract infection'}]]


###5 : Named Entity Recognition (NER) with some Domain Knowledge

In [ ]:
#Case : 5
#If want to give some domain knowledge and description in prompt to enhance the output

result = nlp_prompter.fit('ner.jinja',
                          domain      = 'clinical',
                          text_input  = sent,
                          examples    = one_shot,
                          description = "Below Paragraph is from discharge summary of a patient. The Paragraph describes the condition and symptoms of patient.",
                          labels      = ["SYMPTOM", "DISEASE"])

pprint(eval(result['text']))



The patient is a 93-year-old female with a medical history of chronic right hip pain, osteoporosis, hypertension, depression, and chronic atrial fibrillation admitted for evaluation and management of severe nausea and vomiting and urinary tract infection






[[{'E': 'SYMPTOM', 'W': 'chronic right hip pain'},
  {'E': 'DISEASE', 'W': 'osteoporosis'},
  {'E': 'DISEASE', 'W': 'hypertension'},
  {'E': 'SYMPTOM', 'W': 'depression'},
  {'E': 'DISEASE', 'W': 'chronic atrial fibrillation'},
  {'E': 'SYMPTOM', 'W': 'severe nausea and vomiting'},
  {'E': 'DISEASE', 'W': 'urinary tract infection'}]]


In [ ]:
# for more examples check https://github.com/promptslab/Promptify